In [0]:
# creating a monitoring task for raw data gathering and creating dashboard

In [0]:
%sql
select * from mycatalog.hp_prd_data.raw_data limit 2

In [0]:
%sql
select
 processed_date,
 processed_hour,
 http_status_code,
 count(*) as total_rows,
 count(Distinct zipcode) as total_zipcodes
from mycatalog.hp_prd_data.raw_data
group by processed_date, processed_hour, http_status_code
order by processed_date desc

In [0]:
%sql
select
 count(*) as total_rows,
 count(Distinct source_url) as total_sources,
 count(Distinct zipcode) as total_zipcodes
from mycatalog.hp_prd_data.raw_data

In [0]:
# there are some duplicates do deduplication while transforming

In [0]:
# Parsing raw data

In [0]:
import requests
import json

In [0]:
import json
from pyspark.sql.functions import udf
from pyspark.sql.types import MapType, StringType

def parse_json_text(text: str):
    if text is None:
        return None
    json_text = text[4:]
    try:
        data = json.loads(json_text)
    except Exception as e:
        # You can log the error or skip
        data = None
    return data


In [0]:
import json
def parser(text: str):
    json_text = text[4:]
    try:
        data = json.loads(json_text)
    except Exception as e:
        print("unable to parse the text error ", e)
        print("response length ", len(text))
        data = None
    return data

In [0]:
df = spark.sql("select raw_json from mycatalog.hp_prd_data.raw_data limit 1").collect()[0]
response_text = df['raw_json']

In [0]:
response_text

In [0]:
raw_json = parser(response_text)

In [0]:
raw_json

In [0]:
# now we need to create pyspark code
# how to load required data from the upstream
# db we may have the options to load only.

In [0]:
# target table: mycatalog.hp_prd_data.parsed_data
try:
    last_processed_timestamp = spark

In [0]:
# timestamp calculation
    # last_processed_timestamp = spark.sql("select max(ingestion_timestamp) from mycatalog.hp_prd_data.parsed_data").collect()[0]['max']



In [0]:
def process(row):
    try:
        return {
            "mls#": row.get("mlsId", {}).get("value"),
            "mls_status": row.get("mlsStatus"),
            "house_price": row.get("price", {}).get("value"),
            "hoa": row.get("hoa", {}).get("value"),
            "sqft": row.get("sqft", {}).get("value"),
            "price_per_sqft": row.get("pricePerSqFt", {}).get("value"),
            "lot_size": row.get("lotSize", {}).get("value"),
            "beds": row.get("beds"),
            "baths": row.get("baths"),
            "fullbaths": row.get("fullbaths"),
            "partialBaths": row.get("partialBaths"),
            "location": row.get("location", {}).get("value"),
            "stories": row.get("stories"),
            "latitude": row.get("latLong", {}).get("value", {}).get("latitude"),  # noqa
            "longitude": row.get("latLong", {}).get("value", {}).get("longitude"),  # noqa
            "streetLine": row.get("streetLine", {}).get("value"),
            "city": row.get("city"),
            "state": row.get("state"),
            "zip": row.get("zip"),
            "postalCode": row.get("postalCode", {}).get("value"),
            "countryCode": row.get("countryCode"),
            # "soldDate": row.get("soldDate", {}).get("$numberLong"),
            "searchStatus": row.get("searchStatus"),
            "propertyType": row.get("propertyType"),
            "uiPropertyType": row.get("uiPropertyType"),
            "listingType": row.get("listingType"),
            "propertyId": row.get("propertyId"),
            "listingId": row.get("listingId"),
            "dataSourceId": row.get("dataSourceId"),
            "marketId": row.get("marketId"),
            "yearBuilt": row.get("yearBuilt", {}).get("value"),
            "openHouseStartFormatted": row.get("openHouseStartFormatted"),
            "openHouseEventName": row.get("openHouseEventName"),
            "url": row.get("url"),
            "isHot": row.get("isHot"),
            "hasVirtualTour": row.get("hasVirtualTour"),
            "hasVideoTour": row.get("hasVideoTour"),
            "has3DTour": row.get("has3DTour"),
            "isActiveKeyListing": row.get("isActiveKeyListing"),
            "isNewConstruction": row.get("isNewConstruction"),
            "listingRemarks": row.get("listingRemarks"),
            "scanUrl": row.get("scanUrl"),
            "posterFrameUrl": row.get("posterFrameUrl")
            }
    except KeyError as e:
        print("Key error occured when parsing single doc:", e)
        print("original doc: ------------\n ", row)
        print("\n\n")
    except AttributeError as e:
        print("AttributeError error occured when parsing single doc:", e)
        print("original doc: ------------\n ", row)
        print("\n\n")

In [0]:
raw_json.keys()

In [0]:
data = raw_json.get("payload").get("originalHomes").get("homes")[0]

In [0]:
data

In [0]:
process(data)

In [0]:
# okay we need to convert all those things to pyspark codes

In [0]:
import json
from pyspark.sql.functions import udf
from pyspark.sql.types import MapType, StringType

def parse_json_text(text: str):
    if text is None:
        return None
    json_text = text[4:]
    try:
        data = json.loads(json_text)
    except Exception as e:
        # You can log the error or skip
        data = None
    return data


In [0]:
parse_json_udf = udf(parse_json_text, MapType(StringType(), StringType()))

In [0]:
df = spark.sql("select raw_json from mycatalog.hp_prd_data.raw_data limit 1")

In [0]:
df.show()

In [0]:
from pyspark.sql.functions import col

In [0]:
df_exp = df.withColumn("Json_file", parse_json_text(col("raw_json")))

In [0]:
# Testing GPT code

In [0]:
df = spark.sql("SELECT raw_json FROM mycatalog.hp_prd_data.raw_data")

In [0]:
import json
from pyspark.sql.functions import udf
from pyspark.sql.types import *
# Define schema of one flattened home
home_schema = StructType([
    StructField("mls#", StringType()),
    StructField("mls_status", StringType()),
    StructField("house_price", LongType()),
    StructField("hoa", StringType()),                  # None or str
    StructField("sqft", StringType()),                 # Can be None or float
    StructField("price_per_sqft", StringType()),
    StructField("lot_size", StringType()),
    StructField("beds",StringType()),
    StructField("baths", StringType()),
    StructField("fullbaths", StringType()),
    StructField("partialBaths", StringType()),
    StructField("location", StringType()),
    StructField("stories", StringType()),              # stories is 1.0, so use DoubleType
    StructField("latitude", DoubleType()),
    StructField("longitude", DoubleType()),
    StructField("streetLine", StringType()),
    StructField("city", StringType()),
    StructField("state", StringType()),
    StructField("zip", StringType()),
    StructField("postalCode", StringType()),
    StructField("countryCode", StringType()),
    StructField("searchStatus", IntegerType()),        # 1
    StructField("propertyType", IntegerType()),        # 6
    StructField("uiPropertyType", IntegerType()),      # 1
    StructField("listingType", IntegerType()),         # 1
    StructField("propertyId", LongType()),             # 38910260
    StructField("listingId", LongType()),              # 205114738
    StructField("dataSourceId", IntegerType()),
    StructField("marketId", IntegerType()),
    StructField("yearBuilt", IntegerType()),
    StructField("openHouseStartFormatted", StringType()),
    StructField("openHouseEventName", StringType()),
    StructField("url", StringType()),
    StructField("isHot", BooleanType()),
    StructField("hasVirtualTour", BooleanType()),
    StructField("hasVideoTour", BooleanType()),
    StructField("has3DTour", BooleanType()),
    StructField("isActiveKeyListing", BooleanType()),
    StructField("isNewConstruction", BooleanType()),
    StructField("listingRemarks", StringType()),
    StructField("scanUrl", StringType()),
    StructField("posterFrameUrl", StringType())
])

# Define UDF that returns an array of flattened homes
def extract_listings(text):
    results = []
    try:
        json_text = text[4:]
        data = json.loads(json_text)
        homes = data.get("payload", {}).get("originalHomes", {}).get("homes", [])
        
        for row in homes:
            try:
                results.append({
                    "mls#": row.get("mlsId", {}).get("value"),
                    "mls_status": row.get("mlsStatus"),
                    "house_price": row.get("price", {}).get("value"),
                    "hoa": row.get("hoa", {}).get("value") if "hoa" in row and "value" in row["hoa"] else None,
                    "sqft": row.get("sqFt", {}).get("value") if "sqFt" in row else None,  # FIXED
                    "price_per_sqft": row.get("pricePerSqFt", {}).get("value"),
                    "lot_size": row.get("lotSize", {}).get("value"),
                    "beds": row.get("beds"),
                    "baths": row.get("baths"),
                    "fullbaths": row.get("fullbaths"),
                    "partialBaths": row.get("partialBaths"),
                    "location": row.get("location", {}).get("value"),
                    "stories": row.get("stories"),
                    "latitude": row.get("latLong", {}).get("value", {}).get("latitude"),
                    "longitude": row.get("latLong", {}).get("value", {}).get("longitude"),
                    "streetLine": row.get("streetLine", {}).get("value"),
                    "city": row.get("city"),
                    "state": row.get("state"),
                    "zip": row.get("zip"),
                    "postalCode": row.get("postalCode", {}).get("value"),
                    "countryCode": row.get("countryCode"),
                    "searchStatus": row.get("searchStatus"),
                    "propertyType": row.get("propertyType"),
                    "uiPropertyType": row.get("uiPropertyType"),
                    "listingType": row.get("listingType"),
                    "propertyId": row.get("propertyId"),
                    "listingId": row.get("listingId"),
                    "dataSourceId": row.get("dataSourceId"),
                    "marketId": row.get("marketId"),
                    "yearBuilt": row.get("yearBuilt", {}).get("value"),
                    "openHouseStartFormatted": row.get("openHouseStartFormatted"),
                    "openHouseEventName": row.get("openHouseEventName"),
                    "url": row.get("url"),
                    "isHot": row.get("isHot"),
                    "hasVirtualTour": row.get("hasVirtualTour"),
                    "hasVideoTour": row.get("hasVideoTour"),
                    "has3DTour": row.get("has3DTour"),
                    "isActiveKeyListing": row.get("isActiveKeyListing"),
                    "isNewConstruction": row.get("isNewConstruction"),
                    "listingRemarks": row.get("listingRemarks"),
                    "scanUrl": row.get("scanUrl"),
                    "posterFrameUrl": row.get("posterFrameUrl")
                })
            except Exception as e:
                print("we have an error ", e)
                continue
            print("PRICE BLOCK:", row.get("price"))    
    except Exception as e:
        pass
    return results
# Register as UDF
extract_udf = udf(extract_listings, ArrayType(home_schema))


In [0]:
df_extracted = df.withColumn("homes", extract_udf("raw_json"))

In [0]:
df_final = df_extracted.selectExpr("explode(homes) as home").select("home.*")

In [0]:
%sql
drop table mycatalog.hp_prd_data.dev_parsed_data

In [0]:
df_final.write.mode("overwrite").saveAsTable("mycatalog.hp_prd_data.dev_parsed_data")

In [0]:
%sql
select * from mycatalog.hp_prd_data.dev_parsed_data limit 10

In [0]:
%sql
select * from mycatalog.hp_prd_data.dev_parsed_data where fullbaths is not null limit 10